## Monitoring simluations in SDPySim

### SImulation input data

In [9]:
import pandas as pd

df = pd.read_csv("jobs.csv")
df.head()

,Unnamed: 0,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
0,0,539,116.381625,1.0,0.005405,1,1,283,0
1,1,7448,73.597015,0.5,0.007864,9,23,67,0
2,2,7448,87.380952,0.5,0.007848,9,24,21,0
3,3,7448,21.176471,0.5,0.016007,9,25,17,0
4,4,7448,2.000000,0.5,0.005405,9,26,1,0


#### Simulation output

* Submission time: When it is submitted in the simulation time frame (i.e. greater the value, the further down the track the job is submitted)

* Duration: fairly self-explanatory 

* CPU: (I think) this is the percentage of the CPU used by the task - we can treat it like that, at any rate

* job/task_id: IDs of the job and task, respectively

* instances_num: this is the number of instances of this task that exist in the job. 

Is disk ever greater than 0? 

In [9]:
df[(df.disk>0)] # The anwer is a resounding 'No'. 

,Unnamed: 0,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk


In [27]:
# The first job that will be submitted 
df[(df.submit_time==0)] 

,Unnamed: 0,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
2831,2831,0,131.964467,0.5,0.008935,1750,9550,197,0
2832,2832,0,58.142857,0.5,0.007895,1750,9551,77,0
2833,2833,0,109.452675,0.5,0.008997,1750,9552,243,0
2834,2834,0,38.068966,0.5,0.016007,1750,9553,29,0
2835,2835,0,30.615385,0.5,0.013356,1750,9554,13,0
2836,2836,0,36.876712,0.5,0.013356,1750,9555,73,0
2837,2837,0,25.487179,0.5,0.010706,1750,9556,39,0
2838,2838,0,11.000000,0.5,0.008935,1750,9557,2,0
2839,2839,0,3.000000,0.5,0.004214,1750,9558,1,0
2840,2840,0,154.528736,0.5,0.008397,1750,9559,87,0


In [29]:
time0df = df[(df.submit_time==0)] 
time0df.to_csv("jobs_1750.csv")
checkdf = pd.read_csv("jobs_1750.csv")
checkdf.head()

,Unnamed: 0,Unnamed: 0.1,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
0,2831,2831,0,131.964467,0.5,0.008935,1750,9550,197,0
1,2832,2832,0,58.142857,0.5,0.007895,1750,9551,77,0
2,2833,2833,0,109.452675,0.5,0.008997,1750,9552,243,0
3,2834,2834,0,38.068966,0.5,0.016007,1750,9553,29,0
4,2835,2835,0,30.615385,0.5,0.013356,1750,9554,13,0


Let's split up a couple of jobs from the main job file to get an insight to how the system handles different jobs. 

In [30]:
# This is the two jobs that follow the first job (id 1750)
df[(df.submit_time<100)&(df.submit_time>0)&((df.job_id==1751)|(df.job_id==5970))].head()

,Unnamed: 0,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
2842,2842,54,46.761905,0.5,0.007972,1751,9561,189,0
2843,2843,54,65.311111,0.5,0.013356,1751,9562,45,0
2844,2844,54,93.736842,0.5,0.013356,1751,9563,57,0
2845,2845,54,23.923077,0.5,0.013356,1751,9564,91,0
2846,2846,54,13.000000,0.5,0.008702,1751,9565,2,0


In [33]:
morejobsdf = df[(df.submit_time<100)&(df.submit_time>0)&((df.job_id==1751)|(df.job_id==5970))]
morejobsdf.to_csv("jobs_1751-5970.csv")
checkdf = pd.read_csv("jobs_1751-5970.csv")
checkdf.head()

,Unnamed: 0,Unnamed: 0.1,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
0,2842,2842,54,46.761905,0.5,0.007972,1751,9561,189,0
1,2843,2843,54,65.311111,0.5,0.013356,1751,9562,45,0
2,2844,2844,54,93.736842,0.5,0.013356,1751,9563,57,0
3,2845,2845,54,23.923077,0.5,0.013356,1751,9564,91,0
4,2846,2846,54,13.000000,0.5,0.008702,1751,9565,2,0


### Running Simluations

In [36]:
import sys
sys.path.append('..')

from core.machine import MachineConfig
from playground.utils.episode import Episode
from playground.algorithm.random_algorithm import RandomAlgorithm
from playground.utils.csv_reader import CSVReader
import os
import time
import numpy as np
from playground.utils.tools import multiprocessing_run, average_completion, average_slowdown

machines_number = 5
jobs_len = 1
# n_iter = 50 # Original code was 100
# n_episode = 12
jobs_csv = "jobs_1750.csv"
machine_configs = [MachineConfig(64, 1, 1) for i in range(machines_number)]
csv_reader = CSVReader(jobs_csv)
jobs_configs = csv_reader.generate(0, jobs_len)
tic = time.time()
algorithm = RandomAlgorithm()
episode = Episode(machine_configs, jobs_configs, algorithm, 'event_{0}.trace'.format(jobs_len) )
episode.run()
print("Randome Algorithm")
print(episode.env.now, time.time() - tic, average_completion(episode), average_slowdown(episode))


[1750.0]
Jobs number:  1
Tasks number: 11
Task instances number mean:  84.9090909090909
Task instances number std 79.68398327193893
Task instances cpu mean:  0.5
Task instances cpu std:  0.0
Task instances memory mean:  0.009669336069822145
Task instances memory std:  0.0017946966733641378
Task instances duration mean:  97.95074946466808
Task instances duration std:  37.77646693750589
Randome Algorithm
240 0.17247438430786133 98.20215289346557 1.3788972245178241


In [43]:
jobs_len = 2
# n_iter = 50 # Original code was 100
# n_episode = 12
jobs_csv = "jobs_1751-5970.csv"
machine_configs = [MachineConfig(64, 1, 1) for i in range(machines_number)]
csv_reader = CSVReader(jobs_csv)
jobs_configs = csv_reader.generate(0, jobs_len)
tic = time.time()
algorithm = RandomAlgorithm()
episode = Episode(machine_configs, jobs_configs, algorithm, 'event_{0}.trace'.format(jobs_len) )
episode.run()
print("Randome Algorithm")
print(episode.env.now, time.time() - tic, average_completion(episode), average_slowdown(episode))


[1751.0, 5970.0]
Jobs number:  2
Tasks number: 26
Task instances number mean:  42.15384615384615
Task instances number std 44.45455095478657
Task instances cpu mean:  0.5
Task instances cpu std:  0.0
Task instances memory mean:  0.01074061905020121
Task instances memory std:  0.0022354262673036193
Task instances duration mean:  83.33394160583941
Task instances duration std:  45.9868417391126
Randome Algorithm
302 0.2981283664703369 113.49287250896188 2.0811908539358375


### Results of running simulation

In [50]:
import json 

fp_trace = open('event_1.trace')
trace = json.load(fp_trace) # Trace is a list of dictionaries
fp_trace.close()

In [47]:
trace[0] # First timestamp, nothing much is happening

{'timestamp': 0,
 'cluster_state': {'arrived_jobs': 0,
  'unfinished_jobs': 0,
  'unfinished_job_ids': '[]',
  'finished_jobs': 0,
  'finished_job_ids': '[]',
  'unfinished_tasks': 0,
  'finished_tasks': 0,
  'running_task_instances': 0,
  'machine_states': [{'id': 10,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 1.0,
    'memory': 1.0,
    'disk': 1.0,
    'running_task_instances': 0,
    'finished_task_instances': 0},
   {'id': 11,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 1.0,
    'memory': 1.0,
    'disk': 1.0,
    'running_task_instances': 0,
    'finished_task_instances': 0},
   {'id': 12,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 1.0,
    'memory': 1.0,
    'disk': 1.0,
    'running_task_instances': 0,
    'finished_task_instances': 0},
   {'id': 13,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 1.0,
    'memory': 1.

In [3]:
trace[1]['cluster_state']

{'arrived_jobs': 1,
 'unfinished_jobs': 1,
 'unfinished_job_ids': '[1750.0]',
 'finished_jobs': 0,
 'finished_job_ids': '[]',
 'unfinished_tasks': 11,
 'finished_tasks': 0,
 'running_task_instances': 0,
 'machine_states': [{'id': 0,
   'cpu_capacity': 64,
   'memory_capacity': 1,
   'disk_capacity': 1,
   'cpu': 1.0,
   'memory': 1.0,
   'disk': 1.0,
   'running_task_instances': 0,
   'finished_task_instances': 0},
  {'id': 1,
   'cpu_capacity': 64,
   'memory_capacity': 1,
   'disk_capacity': 1,
   'cpu': 1.0,
   'memory': 1.0,
   'disk': 1.0,
   'running_task_instances': 0,
   'finished_task_instances': 0},
  {'id': 2,
   'cpu_capacity': 64,
   'memory_capacity': 1,
   'disk_capacity': 1,
   'cpu': 1.0,
   'memory': 1.0,
   'disk': 1.0,
   'running_task_instances': 0,
   'finished_task_instances': 0},
  {'id': 3,
   'cpu_capacity': 64,
   'memory_capacity': 1,
   'disk_capacity': 1,
   'cpu': 1.0,
   'memory': 1.0,
   'disk': 1.0,
   'running_task_instances': 0,
   'finished_task_ins

In [30]:
len(df[df.job_id==1750]) # This confirms the above information - Job_id is 1750, with 11 Tasks

11

In [36]:
df[df.job_id==1750]

,Unnamed: 0,submit_time,duration,cpu,memory,job_id,task_id,instances_num,disk
2831,2831,0,131.964467,0.5,0.008935,1750,9550,197,0
2832,2832,0,58.142857,0.5,0.007895,1750,9551,77,0
2833,2833,0,109.452675,0.5,0.008997,1750,9552,243,0
2834,2834,0,38.068966,0.5,0.016007,1750,9553,29,0
2835,2835,0,30.615385,0.5,0.013356,1750,9554,13,0
2836,2836,0,36.876712,0.5,0.013356,1750,9555,73,0
2837,2837,0,25.487179,0.5,0.010706,1750,9556,39,0
2838,2838,0,11.000000,0.5,0.008935,1750,9557,2,0
2839,2839,0,3.000000,0.5,0.004214,1750,9558,1,0
2840,2840,0,154.528736,0.5,0.008397,1750,9559,87,0


In [38]:
trace[3]

{'timestamp': 3,
 'cluster_state': {'arrived_jobs': 1,
  'unfinished_jobs': 1,
  'unfinished_job_ids': '[1750.0]',
  'finished_jobs': 0,
  'finished_job_ids': '[]',
  'unfinished_tasks': 11,
  'finished_tasks': 0,
  'running_task_instances': 499,
  'machine_states': [{'id': 0,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 0.2265625,
    'memory': 0.004555690730722607,
    'disk': 1.0,
    'running_task_instances': 99,
    'finished_task_instances': 0},
   {'id': 1,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 0.203125,
    'memory': 0.0004400383092177957,
    'disk': 1.0,
    'running_task_instances': 102,
    'finished_task_instances': 0},
   {'id': 2,
    'cpu_capacity': 64,
    'memory_capacity': 1,
    'disk_capacity': 1,
    'cpu': 0.2265625,
    'memory': 0.004602283022286888,
    'disk': 1.0,
    'running_task_instances': 99,
    'finished_task_instances': 0},
   {'id': 3,
    'cpu_capacity': 64,
    

In [43]:
trace[3]['cluster_state']['machine_states']

[{'id': 0,
  'cpu_capacity': 64,
  'memory_capacity': 1,
  'disk_capacity': 1,
  'cpu': 0.2265625,
  'memory': 0.004555690730722607,
  'disk': 1.0,
  'running_task_instances': 99,
  'finished_task_instances': 0},
 {'id': 1,
  'cpu_capacity': 64,
  'memory_capacity': 1,
  'disk_capacity': 1,
  'cpu': 0.203125,
  'memory': 0.0004400383092177957,
  'disk': 1.0,
  'running_task_instances': 102,
  'finished_task_instances': 0},
 {'id': 2,
  'cpu_capacity': 64,
  'memory_capacity': 1,
  'disk_capacity': 1,
  'cpu': 0.2265625,
  'memory': 0.004602283022286888,
  'disk': 1.0,
  'running_task_instances': 99,
  'finished_task_instances': 0},
 {'id': 3,
  'cpu_capacity': 64,
  'memory_capacity': 1,
  'disk_capacity': 1,
  'cpu': 0.234375,
  'memory': 0.005223513576476341,
  'disk': 1.0,
  'running_task_instances': 98,
  'finished_task_instances': 0},
 {'id': 4,
  'cpu_capacity': 64,
  'memory_capacity': 1,
  'disk_capacity': 1,
  'cpu': 0.2109375,
  'memory': 0.003251106566925033,
  'disk': 1.0,


We can map these machine states to their respective time stamps to show system load over time (I think it will be pretty subscribed as we are basically loading the states) - from sneaking around the event trace file it peaks at the later stages of the simulation. 

In [51]:
cpu_timestamp = []

for timestamp in trace:
    id_list = [100-x['cpu']*100 for x in timestamp['cluster_state']['machine_states']] #get the CPU_id and usage value
    cpu_timestamp.append(id_list)

cpu_timestamp[3]

[77.34375, 81.25, 78.90625, 81.25, 72.65625]

In [ ]:
jobs_running_timestamp = []

In [52]:
import numpy as np
import matplotlib.pyplot as plt

np_timestamp = np.array(cpu_timestamp)
np_timestamp = np_timestamp.transpose()
x = [x for x in range(np_timestamp.shape[1])]
y = np.vstack(np_timestamp)

# fig, ax = plt.subplots()
# ax.stackplot(x, y1, y2, y3, labels=labels)
# ax.legend(loc='upper left')
# plt.show()
labels=['cpu_{0}'.format(str(x)) for x in range(len(np_timestamp))]
fig, ax = plt.subplots()
ax.set(xlabel="Timestamp", ylabel="Cumulative System usage (%/CPU)",
       title="Cumulative CPU usage of 5 machine cluster over duration of 1 job (11 task) simulation")
# legend = ax.legend(loc='upper right', shadow=True)
# for machine in np_timestamp:
#     ax.stackplot(x,machine)
ax.stackplot(x, y, labels=labels)
ax.legend(loc='upper right')
fig.set_size_inches(8.5,8.5)
# plt.show()
plt.savefig("plot_1.png",dpi=200)
plt.close(fig)

In [84]:
y

array([[100.     , 100.     ,  22.65625, ..., 100.     , 100.     ,
        100.     ],
       [100.     , 100.     ,  20.3125 , ..., 100.     , 100.     ,
        100.     ],
       [100.     , 100.     ,  22.65625, ...,  97.65625,  97.65625,
        100.     ],
       [100.     , 100.     ,  23.4375 , ...,  98.4375 ,  98.4375 ,
        100.     ],
       [100.     , 100.     ,  21.09375, ...,  99.21875,  99.21875,
        100.     ]])

In [ ]:
# NOTE: Add number of running tasks in plot at a given timestamp